### Fonctions utilitaires

Compléter en ajoutant les fonctions de padding et de découpage en bloc.

In [1]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.array(toLoad)

test = Image.open("test2.png")
test1 = load("test2.png")

def psnr(original, compressed):
    compressed_resized = np.resize(compressed, original.shape)
    mse = np.mean((original.astype(int) - compressed_resized) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr





In [2]:
def pading_verif(matrice):
    lignes, colonnes = matrice.size
    
    # Modifie la largeur et la hauteur au multiple de 8 le plus proche
    while lignes % 4 != 0:
        lignes = lignes + 1
    while colonnes % 4 != 0:
        colonnes = colonnes + 1
    
    # Cela crée une nouvelle image avec une largeur et une hauteur ajustées, puis la remplit de noir autour
    result = Image.new(matrice.mode, (lignes, colonnes), (0, 0, 0))
    
    # Colle l'image d'origine dans la nouvelle image également
    result.paste(matrice, (0, 0))
    result.save("test2.png")

    return result

print(pading_verif(test))

def decouper_en_blocs(matrice):
    lignes, colonnes, _ = matrice.shape
    blocs = []

    # Calculer le nombre de lignes et de colonnes à prendre en compte pour les blocs
    nb_lignes_bloc = lignes // 4
    nb_colonnes_bloc = colonnes // 4

    # Parcourir la matrice et découper en blocs 4x4
    for i in range(nb_lignes_bloc):
        for j in range(nb_colonnes_bloc):
            bloc = matrice[i*4:(i+1)*4, j*4:(j+1)*4]
            blocs.append(bloc)

    return blocs




# Découper la matrice en blocs 4x4
blocs = decouper_en_blocs(test1)

# Afficher le nombre total de blocs et la première matrice de bloc
print("Nombre total de blocs :", len(blocs))
print("Premier bloc :\n", blocs[0])


<PIL.Image.Image image mode=RGB size=500x364 at 0x1D9EB0F5DD0>
Nombre total de blocs : 11375
Premier bloc :
 [[[232 235 233]
  [232 235 233]
  [233 236 234]
  [233 236 234]]

 [[235 238 235]
  [235 238 235]
  [235 238 236]
  [236 239 236]]

 [[235 238 235]
  [235 238 236]
  [236 238 236]
  [236 239 236]]

 [[235 238 235]
  [236 238 235]
  [236 239 236]
  [236 239 236]]]


In [3]:
def reconstruire_image(blocs):
    # Obtenir le nombre total de blocs et la taille de l'image
    nb_blocs = len(blocs)
    nb_lignes = nb_blocs * 4
    nb_colonnes = 4
    
    # Créer une matrice vide pour l'image reconstruite
    image_reconstruite = np.zeros((nb_lignes, nb_colonnes, 3), dtype=np.uint8)
    
    # Placer chaque bloc dans l'image reconstruite
    for i, bloc in enumerate(blocs):
        ligne_debut = i * 4
        ligne_fin = ligne_debut + 4
        image_reconstruite[ligne_debut:ligne_fin, :, :] = bloc
    
    return image_reconstruite

    
    
# Fonction de test pour vérifier si l'image reconstruite correspond à l'image d'origine
def tester_fonctions(matrice):
    
    blocs = decouper_en_blocs(matrice)
    
    image_reconstruite = reconstruire_image(blocs)

    print(psnr(matrice,image_reconstruite))
    

# Tester les fonctions
tester_fonctions(test1)
print(reconstruire_image(blocs))
image_pil = Image.fromarray(reconstruire_image(blocs))
image_pil.show()


22.110891303218803
[[[232 235 233]
  [232 235 233]
  [233 236 234]
  [233 236 234]]

 [[235 238 235]
  [235 238 235]
  [235 238 236]
  [236 239 236]]

 [[235 238 235]
  [235 238 236]
  [236 238 236]
  [236 239 236]]

 ...

 [[233 235 239]
  [234 237 241]
  [231 234 238]
  [228 231 234]]

 [[232 235 238]
  [231 234 237]
  [226 229 232]
  [227 230 233]]

 [[228 231 234]
  [226 230 232]
  [225 228 230]
  [226 230 233]]]


### Utilitaires pour le calcul des palettes et des patchs

Fonctions auxiliaires pour manipuler les palettes, les pixels et les blocs. 

In [4]:
def tronque(n, p):
    return n >> p << p

def construire_palette(a, b):
    palette = [
        np.round(a), 
        np.round(2 * a / 3 + b / 3),
        np.round(a / 3 + 2 * b / 3),np.round(b)
        ]
    return palette

# Exemple :
a = np.array([255, 100, 0])  # Couleur a
b = np.array([50, 100, 255])  # Couleur b

# Construire la palette à partir des couleurs a et b
palette = construire_palette(a, b)

print("Palette :", palette)

def distance_couleur(couleur1, couleur2):
    return np.linalg.norm(couleur1 - couleur2)

def couleur_plus_proche(pixel, palette):
    distance_min = float('inf')  # Initialisation de la distance minimale à l'infini
    couleur_proche = None  # Initialisation de la couleur la plus proche à None

    # Parcourir chaque couleur de la palette
    for i, couleur in enumerate(palette):
        # Calculer la distance entre le pixel et la couleur de la palette
        distance = distance_couleur(pixel, couleur)
        
        # Mettre à jour la couleur la plus proche si la distance est plus petite que la distance minimale actuelle
        if distance < distance_min:
            distance_min = distance
            couleur_proche = i  # Enregistrer l'indice de la couleur dans la palette
    
    return couleur_proche

pixel = np.array([50, 100, 200])

# Trouver la couleur de la palette la plus proche du pixel donné
indice_couleur_proche = couleur_plus_proche(pixel, palette)
print("Indice de la couleur la plus proche dans la palette :", indice_couleur_proche)

Palette : [array([255, 100,   0]), array([187., 100.,  85.]), array([118., 100., 170.]), array([ 50, 100, 255])]
Indice de la couleur la plus proche dans la palette : 3


### Les méthodes de choix des couleurs de la palette


On peut essayer plusieurs stratégies pour calculer les deux couleurs a et b:
* Calculer la couleur minimale et maximale
* Prendre des couleurs moyennes (calculer moyenne et variance pour trouver les meilleurs valeurs) 
* Calculer 10 valeurs au hasard et prendre la meilleure
* Partir d'un choix et perturber pour améliorer le résultat: on teste la meilleure modification de 16 d'un des canaux d'une des couleurs

In [5]:
def trouver_couleurs_min_max(patch):
    # Calculer les valeurs minimales et maximales pour chaque canal
    min_canal = np.min(patch, axis=(0, 1))
    max_canal = np.max(patch, axis=(0, 1))
    
    # Formater les couleurs au bon format en utilisant la fonction tronque
    a = np.array([tronque(min_canal[0], 3), tronque(min_canal[1], 2), tronque(min_canal[2], 3)], dtype=np.uint8)
    b = np.array([tronque(max_canal[0], 3), tronque(max_canal[1], 2), tronque(max_canal[2], 3)], dtype=np.uint8)
    
    return a, b

def compress_patch(patch):
    # Calcul de la moyenne et de l'écart-type pour chaque canal de couleur (rouge, vert, bleu)
    mean_couleur = np.mean(patch, axis=(0, 1))
    std_couleur = np.std(patch, axis=(0, 1))
    
    # Définition des couleurs en fonction de la moyenne et de l'écart-type
    limite_inferieur = np.clip(mean_couleur - std_couleur, 0, 255).astype(int)
    limite_superieur = np.clip(mean_couleur + std_couleur, 0, 255).astype(int)
    
    return limite_inferieur, limite_superieur


a, b = trouver_couleurs_min_max(blocs[0])
limite_inferieur, limite_superieure = compress_patch(blocs[0])

# Affichage des résultats

print("Couleur a :", a)
print("Couleur b :", b)

print("Couleurs en fonction de la moyenne et de l'écart-type (bornes inférieures) :", limite_inferieur)
print("Couleurs en fonction de la moyenne et de l'écart-type (bornes supérieures) :", limite_superieure)


Couleur a : [232 232 232]
Couleur b : [232 236 232]
Couleurs en fonction de la moyenne et de l'écart-type (bornes inférieures) : [233 236 234]
Couleurs en fonction de la moyenne et de l'écart-type (bornes supérieures) : [236 238 236]


### Encodage d'un bloc

Écrire ici le code qui permet d'encoder un bloc et une palette en un entier.

In [6]:
def construire_entier(a, b, palette):
    """Construit l'entier représentant le patch à partir de a, b et des indices de la palette."""
    # Nombre de bits pour les indices
    nb_bits_indices = len(palette) * len(palette[0])
    
    # Conversion des valeurs de a et b en format 5:6:5
    a_565 = ((a >> 3) << 11).astype(np.uint16)
    b_565 = ((b >> 2) << 5).astype(np.uint16)

    
    # Construction de l'entier
    entier = 0
    for i in range(nb_bits_indices):
        indice = int(palette[i // len(palette[0])][i % len(palette[0])])
        entier |= indice << i
    entier|= b_565
    entier|= a_565
    
    return entier

# Construire l'entier représentant le patch
entier_patch = construire_entier(a, b, palette)

print("Entier représentant le patch :", entier_patch)


Entier représentant le patch : [524287 524287 524287]


### Écriture et lecture dans un fichier

In [15]:
# Ouvrir le fichier en mode écriture
with open("informations_image.txt", "w") as f:
    # Écrire le type de fichier
    f.write("BC1\n")
    
    # Écrire les dimensions de l'image (hauteur puis largeur)
    f.write("200 300\n")

    f.write(str(entier_patch[0])+ "\n")


    f.write(str(entier_patch[1])+ "\n")
    
    f.write(str(entier_patch[2])+ "\n")

def lire_fichier_BC1(nom_fichier):
    # Ouvrir le fichier BC1 en mode lecture binaire
    with open(nom_fichier, "rb") as f:
        # Lire toutes les données du fichier
        donnees = f.read()

    # Taille d'un bloc BC1 en octets
    taille_bloc = 8  # Chaque bloc est codé sur 8 octets dans un fichier BC1

    # Calculer le nombre total de blocs dans le fichier
    nb_blocs = len(donnees) // taille_bloc

    # Créer une liste pour stocker les entiers représentant chaque bloc
    blocs_entiers = []

    # Parcourir les données brutes et reconstruire chaque bloc
    for i in range(nb_blocs):
        # Début de la partie de données du bloc actuel
        debut = i * taille_bloc
        # Fin de la partie de données du bloc actuel
        fin = (i + 1) * taille_bloc
        # Extraire les données du bloc actuel
        bloc = donnees[debut:fin]

        # Convertir les données du bloc en un entier et l'ajouter à la liste
        bloc_entier = int.from_bytes(bloc, byteorder='little')  # little-endian
        blocs_entiers.append(bloc_entier)

    return blocs_entiers

# Utilisation de la fonction pour lire un fichier BC1 et obtenir la liste des entiers représentant chaque bloc
blocs_entiers = lire_fichier_BC1("a.txt")
print(blocs_entiers)

def couleur_paire_et_indice(nombre):
    # Division de l'entier en deux parties
    parti_couleur = nombre >> 16  # Les 16 bits de poids fort
    parti_indice = nombre & 0xFFFF  # Les 16 bits de poids faible
    
    # Création de la paire de couleurs
    couleur1= [(parti_couleur >> 8) & 0xFF, (parti_couleur >> 16) & 0xFF, (parti_couleur >> 24) & 0xFF]
    couleur2 = [parti_couleur & 0xFF, (parti_couleur >> 8) & 0xFF, (parti_couleur >> 16) & 0xFF]
    
    # Création du tableau d'indices
    indice = np.unpackbits(np.array([parti_indice], dtype=np.uint16).view(np.uint8))[8:]
    
    return couleur1, couleur2, indice

def reconstruire_patch(couleur1, couleur2, indice):
    # Création d'un tableau vide pour le patch
    patch = np.zeros((8, 8, 3), dtype=np.uint8)
    
    # Remplissage du patch avec les couleurs en fonction des indices
    for i, idx in enumerate(indice):
        if idx == 0:
            patch[i // 8, i % 8] = couleur1
        else:
            patch[i // 8, i % 8] = couleur2
            
    return patch


nombre = entier_patch[0]
couleur1, couleur2, indice = couleur_paire_et_indice(nombre)
patch = reconstruire_patch(couleur1, couleur2, indice)

# Affichage des résultats
print("Paire de couleurs :", couleur1, couleur2)
print("Indices :", indice)
print("Patch reconstruit :", patch)

[3472567839438680113]
Paire de couleurs : [0, 0, 0] [7, 0, 0]
Indices : [1 1 1 1 1 1 1 1]
Patch reconstruit : [[[7 0 0]
  [7 0 0]
  [7 0 0]
  [7 0 0]
  [7 0 0]
  [7 0 0]
  [7 0 0]
  [7 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


### Fonctions de test

Écrire ici tous vos tests. Chaque fonction écrite doit être testée.

In [9]:
def construire_entier(a, b, indices_palette):
    """Construit l'entier représentant le patch à partir de a, b et des indices de la palette."""
    # Nombre de bits pour les indices
    nb_bits_indices = len(indices_palette) * len(indices_palette[0])
    
    # Conversion des valeurs de a et b en format 5:6:5
    a_565 = (a >> 3) << 11
    b_565 = (b >> 2) << 5
    
    # Construction de l'entier
    entier = 0
    for i in range(nb_bits_indices):
        entier |= (indices_palette[i] << i)
    entier |= b_565
    entier |= a_565
    
    return entier

# Exemple d'utilisation :
a = np.array([100, 200, 50])  # Couleur a
b = np.array([30, 150, 220])  # Couleur b
indices_palette = [1, 0, 3, 2]  # Indices de la palette (exemple)

# Construire l'entier représentant le patch
entier_patch = construire_entier(a, b, indices_palette)

print("Entier représentant le patch :", entier_patch)

def bloc(old_table: List[List[int]]) -> List[List[int]]:
    """
    Applies the Discrete Cosine Transform (DCT) to the input old_table.

    Args:
        old_table (List[List[int]]): The original table to be transformed.

    Returns:
        List[List[int]]: The transformed table after applying the DCT.
    """

    pi = math.pi  # Assigns the value of pi
    # Creates a new table 'new_table' by copying 'old_table'
    new_table = deepcopy(old_table)

    for new_y in range(8):
        # Calculates the cos of an angle and assigns it to the variable 'cy'
        cy = cos(((2 * new_y + 1) * pi) / 16)

        for new_x in range(8):
            # Calculates the cos of an angle and assigns it to the variable 'cx'
            cx = cos(((2 * new_x + 1) * pi) / 16)

            # Calculates the new value of an element in the list using the formula for the DCT
            new_value = sum(
                old_table[old_y][old_x] * cos(((2 * old_y + 1) * new_y * pi) / 16) * cos(
                    ((2 * old_x + 1) * new_x * pi) / 16)
                for old_y in range(8) for old_x in range(8)
            )

            if new_y == 0:
                # Divides the new values by the square root of 2 for the first row
                new_value /= sqrt(2)
            if new_x == 0:
                # Divides the new values by the square root of 2 for the first column
                new_value /= sqrt(2)

            # Multiplies the new value by (0.25 * cx * cy)
            new_value *= (0.25 * cx * cy)
            # Updates the new table with the new value
            new_table[new_y][new_x] = new_value

    return new_table

with open("text.txt", "w") as f:
        height,width = 200 ,300
        f.write("BC1\n")  # Header which indicates the file format
        # Writes the height and width of the image
        f.write(f"{height} {width}\n")

    # Comparer l'image reconstruite avec l'image d'origine
    """if np.array_equal(matrice, image_reconstruite):
        print("Les fonctions sont correctes. L'image n'a pas changé.")
    else:
        print("Les fonctions ne sont pas correctes. L'image a changé.")"""
# Créer une matrice 16x16x3 (trois canaux pour la couleur) avec des valeurs aléatoires
"""matrice = np.random.randint(0, 256, (4, 4, 3), dtype=np.uint8)"""


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 80)